In [1]:
import numpy as np

In [70]:
def GF_table(bc_left: str, bc_right: str, L, truncation):
    # returns the eigenvalue beta_m and the norm Nx

    m = np.arange(1, truncation+1)
    if (bc_left == 'D' and bc_right == 'D'): #11
        beta_m = m*np.pi/L
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'D' and bc_right == 'N'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'N' and bc_right == 'D'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    

def green_function(x, y, x_s, y_s, Lx, Ly, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 50):
    # x = np.arange(0, Lx, step)
    # y = np.arange(0, Ly, step)
    beta_m, Nx = GF_table(bc_e, bc_w, Lx, truncation)
    theta_n, Ny = GF_table(bc_s, bc_n, Ly, truncation)
    summation_array = np.zeros(truncation*truncation)
    
    for m in range(1, truncation+1):
        i = m-1

        X_m = np.sin(x*beta_m[i])
        X_m_s = np.sin(x_s*beta_m[i])
        # print("X_m_s: " + str(X_m_s))
        for n in range(1, truncation+1):
            j= n-1

            Y_n = np.sin(y*theta_n[j])
            Y_n_s = np.sin(y_s*theta_n[j])
            # print("X_m_s: " + str(Y_n_s))

            fraction = 1/((beta_m[i]*beta_m[i]) + (theta_n[j]*theta_n[j]))
            X_mult = (X_m*X_m_s)/Nx
            Y_mult = (Y_n*Y_n_s)/Ny

            # summation_array[i*truncation + j] = fraction * np.dot(X_mult, Y_mult)
            summation_array[i*truncation + j] = fraction * X_mult * Y_mult
    
    sum = np.sum(summation_array)
    # return sum
    return sum

In [78]:
Lx = Ly = 1
x_s = y_s = Lx/2
n_step = 2
step = 0.01 # limit might be after 0.01
# np.linspace(0, L/2, n_step, endpoint=True, retstep=False, dtype=None, axis=0)
# x_in = np.arange(0, L/2, step)
# x_out = np.arange(0, L/2, step)
# y_in = np.arange(0, L/2, step)
# y_out = np.arange(0, L/2, step)
x = np.arange(0, Lx, step)
y = np.arange(0, Ly, step)

green_function(x[1], y[1], x_s, y_s, Lx, Ly, bc_e = "D", bc_n = "D", bc_w = "D", bc_s = "D")
# green_function(x, y, x_s, y_s, L, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 10):

0.0001296981110708539